#### 데이터 준비

In [1]:
import pandas as pd
import numpy as np


In [5]:
data = pd.read_csv('./data/공공보건의료기관현황.csv', encoding='utf-8')
data.head()

,연번,병원 명,설립형태,근거 법령,관계 행정기관,관계 공공단체,심평원 요양기관번호,종별구분,병상수,소재지 우편번호,주소,홈페이지,대표전화,FAX,비고
0,1,강원도 재활병원,시도립,강원도재활병원설치및운영에관한조례,강원도,해당없음,32200641,병원,165,24227,강원도 춘천시 충열로 142번길 24-16,www.grh.or.kr,033-248-7700,033-248-7723,NaN
1,2,강원도 삼척의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100060,종합병원,152,25920,강원도 삼척시 오십천로 418,http://ksmc.or.kr,033-572-1141,033-573-8424,NaN
2,3,강원도 영월의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100078,종합병원,214,26234,강원도 영월군 영월읍 중앙1로 59,http://www.youngwol.org,033-370-9117,033-370-9137,NaN
3,4,강원도 원주의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100086,종합병원,237,26448,강원도 원주시 서원대로 387(개운동),www.kwmc.or.kr,033-760-4500,033-761-5121,NaN
4,5,강원도 강릉의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100159,종합병원,137,25535,강원도 강릉시 경강로 2007(남문동 164-1),http://www.gnmc.or.kr,033-646-6910,033-610-1415,NaN


#### 주소 전처리 - 정제

In [10]:
# 주소 시도 구군 정보 분리

addr = pd.DataFrame(data['주소'].apply(lambda v: v.split()[:2]).tolist(), columns = ('시도','구군'))
addr.head()

,시도,구군
0,강원도,춘천시
1,강원도,삼척시
2,강원도,영월군
3,강원도,원주시
4,강원도,강릉시


In [19]:
addr['시도'].unique()

array(['강원도', '경기도', '경기', '경남', '경상남도', '경상북도', '경산시', '경북', '인천광역시',
       '대구광역시', '전라남도', '대전광역시', '광주광역시', '제주특별자치도', '부산광역시', '전라북도',
       '충북', '서울특별시', '서울시', '부산특별시', '대전시', '충남', '전남', '충청남도', '울산광역시',
       '전북', '천안시', '충청북도'], dtype=object)

In [12]:
# 창원시

addr[addr['시도'] == '창원시']


,시도,구군
27,창원시,의창구
31,창원시,마산합포구3.15대로


In [13]:
addr.iloc[27] = ['경상남도', '창원시']
addr.iloc[31] = ['경상남도', '창원시']

In [15]:
addr[addr['시도'] == '경산시']

,시도,구군
47,경산시,경안로


In [20]:
addr.iloc[47] = ['경상북도', '경산시']

In [21]:
addr[addr['시도'] == '천안시']


,시도,구군
209,천안시,동남구
210,천안시,동남구


In [22]:
addr.iloc[209] = ['충청남도', '천안시']
addr.iloc[210] = ['충청남도', '천안시']


In [23]:
addr['시도'].unique()

array(['강원도', '경기도', '경기', '경남', '경상남도', '경상북도', '경북', '인천광역시', '대구광역시',
       '전라남도', '대전광역시', '광주광역시', '제주특별자치도', '부산광역시', '전라북도', '충북',
       '서울특별시', '서울시', '부산특별시', '대전시', '충남', '전남', '충청남도', '울산광역시', '전북',
       '충청북도'], dtype=object)

In [30]:
addr[addr['시도'] == '부산특별시']
addr.iloc[98] = ['부산광역시', '부산광역시']

#### 경기 - 경기도, 경남 - 경상남도, 부산특별시 - 부산광역시

In [43]:
addr_aliases = {'경기':'경기도', '경남':'경상남도', '경북':'경상북도', '광주':'광주광역시', '대구':'대구광역시', '대전':'대전광역시', '부산':'부산광역시', '서울':'서울특별시', '세종':'세종특별자치시', '울산':'울산광역시', '인천':'인천광역시', '전남':'전라남도', '전북':'전라북도', '제주':'제주특별자치도', '충남':'충청남도', '충북':'충청북도', '세종시':'세종특별자치시', '서울시':'서울특별시', '대전시' : '대전광역시'}

addr['시도'] = addr['시도'].apply(lambda v: addr_aliases.get(v, v))
addr['시도'].unique()


array(['강원도', '경기도', '경상남도', '경상북도', '인천광역시', '대구광역시', '전라남도', '대전광역시',
       '광주광역시', '제주특별자치도', '부산광역시', '전라북도', '충청북도', '서울특별시', '충청남도',
       '울산광역시'], dtype=object)

#### 구군 정리

In [44]:
addr['구군'].unique()

array(['춘천시', '삼척시', '영월군', '원주시', '강릉시', '속초시', '정선군', '수원시', '이천시',
       '안성시', '의정부시', '포천시', '파주시', '용인시', '평택시', '시흥시', '여주시', '남양주시',
       '동두천시', '안산시', '부천시', '통영시', '사천시', '창원시', '김해시', '양산시', '거창군',
       '경산시', '의령군', '포항시', '김천시', '안동시', '울진군', '경주시', '구미시', '영주시',
       '상주시', '문경시', '의성군', '청도군', '고령군', '칠곡군', '봉화군', '울릉군', '부평구',
       '북구', '순천시', '대덕구', '태백시', '동해시', '화성시', '광산구', '남구', '중구', '제주시',
       '서구', '전주시', '진주시', '청주시', '종로구', '성남시', '동구', '화순군', '강동구', '사상구',
       '달서구', '부산광역시', '유성구', '가평군', '양주시', '고양시', '홍천군', '양구군', '청원군',
       '계룡시', '논산시', '함평군', '양평군', '수성구', '달성군', '연수구', '노원구', '기장군',
       '공주시', '강북구', '광진구', '나주시', '창녕군', '목포시', '고흥군', '연제구', '해운대구',
       '동매로', '서초구', '은평구', '중랑구', '강남구', '동작구', '동대문구', '양천구', '성동구',
       '송파구', '울주군', '계양구', '옹진군', '보성군', '광양시', '영광군', '무안군', '진도군',
       '강진군', '곡성군', '여수시', '신안군', '장성군', '완주군', '부안군', '정읍시', '남원시',
       '군산시', '고창군', '진안군', '서귀포시', '천안시', '보령시', '서산시', '서천군', '홍성군',
   

In [45]:
addr[addr['구군'] == '아란13길']


,시도,구군,시도구군,행정구역,count


In [46]:
addr.iloc[75] = ['제주특별자치도', '제주시']

ValueError: Must have equal len keys and value when setting with an iterable

In [47]:
addr[addr['구군'] == '부산광역시']

,시도,구군,시도구군,행정구역,count
98,부산광역시,부산광역시,부산광역시 부산광역시,부산광역시 부산광역시,0


In [48]:
#### 행정구역별 공공보건의료기관 수

In [49]:
addr['행정구역'] = addr.apply(lambda r: r['시도'] + ' ' + r['구군'], axis='columns')
addr.head()

,시도,구군,시도구군,행정구역,count
0,강원도,춘천시,강원도 춘천시,강원도 춘천시,0
1,강원도,삼척시,강원도 삼척시,강원도 삼척시,0
2,강원도,영월군,강원도 영월군,강원도 영월군,0
3,강원도,원주시,강원도 원주시,강원도 원주시,0
4,강원도,강릉시,강원도 강릉시,강원도 강릉시,0


In [50]:
addr['count'] = 0
addr

,시도,구군,시도구군,행정구역,count
0,강원도,춘천시,강원도 춘천시,강원도 춘천시,0
1,강원도,삼척시,강원도 삼척시,강원도 삼척시,0
2,강원도,영월군,강원도 영월군,강원도 영월군,0
3,강원도,원주시,강원도 원주시,강원도 원주시,0
4,강원도,강릉시,강원도 강릉시,강원도 강릉시,0
...,...,...,...,...,...
222,충청북도,제천시,충청북도 제천시,충청북도 제천시,0
223,충청북도,충주시,충청북도 충주시,충청북도 충주시,0
224,충청북도,충주시,충청북도 충주시,충청북도 충주시,0
225,충청북도,영동군,충청북도 영동군,충청북도 영동군,0


In [51]:
addr_group = pd.DataFrame(addr.groupby(['시도','구군','행정구역'], as_index=False).count())
addr_group = addr_group.drop(columns='시도구군')

In [52]:
addr_group

,시도,구군,행정구역,count
0,강원도,강릉시,강원도 강릉시,4
1,강원도,동해시,강원도 동해시,1
2,강원도,삼척시,강원도 삼척시,1
3,강원도,속초시,강원도 속초시,1
4,강원도,양구군,강원도 양구군,1
...,...,...,...,...
135,충청북도,영동군,충청북도 영동군,1
136,충청북도,제천시,충청북도 제천시,1
137,충청북도,청원군,충청북도 청원군,1
138,충청북도,청주시,충청북도 청주시,4


In [53]:
addr_group = addr_group.set_index('행정구역')
addr_group

,시도,구군,count
행정구역,,,
강원도 강릉시,강원도,강릉시,4
강원도 동해시,강원도,동해시,1
강원도 삼척시,강원도,삼척시,1
강원도 속초시,강원도,속초시,1
강원도 양구군,강원도,양구군,1
...,...,...,...
충청북도 영동군,충청북도,영동군,1
충청북도 제천시,충청북도,제천시,1
충청북도 청원군,충청북도,청원군,1
